In [0]:
import pandas as pd
import datetime
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

Подход к решению задачи по прогнозированию покупок беременных. Тестовое задание к хакатону от Х5. Испольнитель Бочаров А.М. скайп bam271074

Прогнозирование детских покупок можно сделать на основании покупок, которые женщина делает во время беременности. Известно, что в период беременности женщина нуждается в витаминах и фолиевой кислоте. Таким образом если женщина возраста от 16 до 40 лет часто покупает овощи, фрукты, овес, постное мясо, молочные изделия. На основании данной информации можно построить модель, обучить ее выявлять данную закономерность и использовать в работе.

In [0]:
pip list

Package                  Version        
------------------------ ---------------
absl-py                  0.9.0          
alabaster                0.7.12         
albumentations           0.1.12         
altair                   4.1.0          
asgiref                  3.2.7          
astor                    0.8.1          
astropy                  4.0.1.post1    
astunparse               1.6.3          
atari-py                 0.2.6          
atomicwrites             1.4.0          
attrs                    19.3.0         
audioread                2.1.8          
autograd                 1.3            
Babel                    2.8.0          
backcall                 0.1.0          
beautifulsoup4           4.6.3          
bleach                   3.1.5          
blis                     0.4.1          
bokeh                    1.4.0          
boto                     2.49.0         
boto3                    1.13.13        
botocore                 1.16.13        
Bottleneck      

### Версии:

- `numpy`: 1.18.4
- `pandas`: 1.0.3
- `sklearn`: 0.21.3

#сгенерируем датасет и сделаем бинарную классификацию. Если в покупках есть молочные продукты, то 1(покупательница может быть беременной), 0(в покупках нет продуктов, которые обычно покупают беременные)


In [0]:
#let s fix random state
value = 42
random.seed(value)
np.random.seed(value)

In [12]:
clients = pd.Series(['Ann     ', 'Alla    ', 'Chloe   ', 'Dorophy ', 'Emma    ', 'Filona  ', 'Gina    ',\
                     'Hilary  ', 'Ira     ', 'Sophy   '],
                    )
print(clients)

0    Ann     
1    Alla    
2    Chloe   
3    Dorophy 
4    Emma    
5    Filona  
6    Gina    
7    Hilary  
8    Ira     
9    Sophy   
dtype: object


In [4]:
#допустим у нас в магазине следующие продукты

products = pd.Series(['Abritos', 'Bananas', 'Onion', 'Sugar', 'Eris', 'Fries', 'Bread', 'Water', 'Cheese', 'Salo',\
                      'Vodka','Beer','Vine','Sweet','Cigar','Salt','Pepper',\
                     'Milk','Kefir','Iogurt','Cream'],
                       index=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'])
print(products)

1     Abritos
2     Bananas
3       Onion
4       Sugar
5        Eris
6       Fries
7       Bread
8       Water
9      Cheese
10       Salo
11      Vodka
12       Beer
13       Vine
14      Sweet
15      Cigar
16       Salt
17     Pepper
18       Milk
19      Kefir
20     Iogurt
21      Cream
dtype: object


In [15]:
#сгенерируем список из 20000 элементов, каждая строка записи будет содержать имя покупателя и покупки, которые от сделал
buy_list =[]
buyer_list=[]
flag_list=[]  #списко для записи есть ли молочные продукты или нет
milk_set=set(['Milk','Kefir','Iogurt','Cream','Cheese'])
print(milk_set)
for i in range(20000):
  buyer = ' '.join(clients.sample(1, replace=True))   #выбираем одного клиента из списка всех клиентов
  #buyer = buyer[5:15]
  pokupky = products.sample(3, replace=False)  #выбираем покупки, которые сделал клиент
  for i in range(len(pokupky)):
    flag='negative'
    if pokupky[i] in milk_set:
      flag = 'positive'   #если в покупках есть молоцные продукты то флаг ПОЗИТИВ
      break
  str_buff=' '.join(pokupky)
  buyer_list.append(buyer)
  buy_list.append(str_buff)
  flag_list.append(flag)
print(buy_list)
print(buyer_list)
print(flag_list)


{'Milk', 'Kefir', 'Cream', 'Iogurt', 'Cheese'}
['Fries Onion Milk', 'Sweet Vine Water', 'Cheese Salt Eris', 'Cigar Eris Salo', 'Water Iogurt Beer', 'Fries Vine Sweet', 'Salo Cream Bananas', 'Bananas Eris Iogurt', 'Bread Onion Cream', 'Eris Pepper Sugar', 'Kefir Beer Water', 'Salt Cheese Cream', 'Iogurt Cream Bananas', 'Beer Water Abritos', 'Vine Milk Salt', 'Cigar Salo Pepper', 'Bananas Kefir Cream', 'Sweet Onion Salt', 'Iogurt Vodka Vine', 'Pepper Cheese Salt', 'Cream Water Fries', 'Vine Onion Sugar', 'Water Salt Bread', 'Vine Salt Pepper', 'Abritos Pepper Bread', 'Kefir Cream Bananas', 'Bread Milk Pepper', 'Sweet Cheese Onion', 'Sweet Fries Bread', 'Fries Pepper Iogurt', 'Fries Sugar Abritos', 'Cheese Vodka Milk', 'Abritos Bread Iogurt', 'Vodka Cheese Salt', 'Vodka Cheese Bananas', 'Onion Vodka Sweet', 'Cream Vine Salo', 'Water Abritos Beer', 'Salo Beer Bread', 'Abritos Salt Salo', 'Fries Cream Salo', 'Abritos Vine Beer', 'Cream Cheese Milk', 'Sweet Milk Kefir', 'Pepper Sweet Cream',

In [16]:
#let s make dataframe
data_df=pd.DataFrame({'Client':buyer_list,'Products':buy_list,'Target':flag_list})
data_df.head(20000)

,Client,Products,Target
0,Sophy,Fries Onion Milk,positive
1,Filona,Sweet Vine Water,negative
2,Sophy,Cheese Salt Eris,positive
3,Alla,Cigar Eris Salo,negative
4,Hilary,Water Iogurt Beer,positive
...,...,...,...
19995,Ann,Cigar Fries Beer,negative
19996,Sophy,Vine Abritos Beer,negative
19997,Chloe,Iogurt Water Bread,positive
19998,Ira,Bananas Salt Iogurt,positive


In [0]:
data_df.to_csv('pokupty20000.csv',index=False)

In [0]:
from google.colab import files
files.download('pokupty20000.csv')